In [ ]:
from extra_data import RunDirectory 
import h5py 
import numpy as np 
%matplotlib widget
import matplotlib.pyplot as plt 

In [ ]:
import xarray

def read_run_sync_trains(r, verbose=True):
    #clear_output(wait=False) 
    #sys.stderr.write(f'Processing run {r}...\n') 
    run = RunDirectory(path='/pnfs/xfel.eu/exfel/archive/XFEL/raw/SPB/202202/p003046/r%04d' % (r))  
    
    # The require_all=True ensures we only get trains which exist for all sources
    sel = run.select([
        ('SPB_XTD9_XGM/XGM/DOOCS:output', 'data.intensitySa1TD'),
        ('SA1_XTD2_XGM/XGM/DOOCS:output', 'data.intensitySa1TD'),
        ('SPB_IRU_MOTORS/MDL/DATA_SELECT', '*'),
        ('SPB_EXP_ZYLA/CAM/1:daqOutput', 'data.image.pixels'),
    ], require_all=True)
    if(verbose):
        sel.info()
    
    intensity_sase1_extra = sel['SPB_XTD9_XGM/XGM/DOOCS:output', 'data.intensitySa1TD']
    intensity_preattenuator_sase1_extra = sel['SA1_XTD2_XGM/XGM/DOOCS:output', 'data.intensitySa1TD']    
    motors_x_extra = sel['SPB_IRU_MOTORS/MDL/DATA_SELECT', 'SPB_IRU_INJMOV_MOTOR_X.actualPosition.value']
    motors_y_extra = sel['SPB_IRU_MOTORS/MDL/DATA_SELECT', 'SPB_IRU_INJMOV_MOTOR_Y.actualPosition.value']
    motors_z_extra = sel['SPB_IRU_MOTORS/MDL/DATA_SELECT', 'SPB_IRU_INJMOV_MOTOR_Z.actualPosition.value']

    # Put the data in xarray
    motor_x = motors_x_extra.xarray()
    motor_y = motors_y_extra.xarray()
    motor_z = motors_z_extra.xarray()
    xgm = intensity_sase1_extra.xarray()
    xgm2 = intensity_preattenuator_sase1_extra.xarray()
    
    directory = '/gpfs/exfel/u/scratch/SPB/202202/p003046/data' 
    agipd_frames_per_train = 202 # number of agipd frames per train
    with h5py.File(directory+'/r%04d_proc_radavg.h5' % (r)) as rad: 
        radavg = rad['entry_1']['radialavg'][:]
        trainIds = rad['entry_1']['trainId'][:]
        q = rad['entry_1']['q'][:]
    
    # Reshape the arrays from the radavg file to make them by train
    radavg = radavg.reshape((-1,agipd_frames_per_train,radavg.shape[1]))
    trainIds = trainIds.reshape((-1,agipd_frames_per_train))
    # Ensure all the trains are the same length
    if (trainIds == trainIds[:,:1]).all() != True:
        raise ValueError        
    if(verbose):
        print("Reshaping of radavg successful:", (trainIds == trainIds[:,:1]).all())
    trainIds = trainIds[:,0]
    
    # Put radavg in an xarray
    radavg = xarray.DataArray(data=radavg, dims=('trainId', 'pulseNr', 'q'), 
                              coords={'trainId': trainIds, 'pulseNr': np.arange(agipd_frames_per_train), 'q': q})

    # Find trains which are common between radavg and the rest
    common_trains = np.intersect1d(intensity_sase1_extra.train_id_coordinates(), trainIds)
    if verbose:
        print("Found %d common train IDs between radavg and the rest" % len(common_trains))

    # Find the indices of the common trainIds and keep only those
    common_train_idx = np.searchsorted(intensity_sase1_extra.train_id_coordinates(),common_trains)
    motor_x = motor_x[common_train_idx]
    motor_y = motor_y[common_train_idx]
    xgm = xgm[common_train_idx]

    # Find the indices of the common trainIds for radavg    
    common_train_idx = np.searchsorted(radavg.coords['trainId'].data,common_trains)
    radavg = radavg[common_train_idx]
    
    train_ids_eq = (motor_x.coords['trainId'].data == radavg.coords['trainId'].data)
    if verbose:
        print("Train IDs all match:", train_ids_eq.all())
    
    return xgm, motor_x, motor_y, motor_z, radavg, xgm2

In [ ]:
def extract_gifs(r, verbose=True):
    run = RunDirectory(path=f'/gpfs/exfel/exp/SPB/202202/p003046/raw/r0{r}')  
    
    # The require_all=True ensures we only get trains which exist for all sources
    sel = run.select([
        ('SPB_XTD9_XGM/XGM/DOOCS:output', 'data.intensitySa1TD'),
        ('SPB_IRU_MOTORS/MDL/DATA_SELECT', '*'),
        ('SPB_EXP_ZYLA/CAM/1:daqOutput', 'data.image.pixels'),
    ], require_all=True)
    if(verbose):
        sel.info()   
    sidemic_extra = sel['SPB_EXP_ZYLA/CAM/1:daqOutput', 'data.image.pixels']
    trains = sidemic_extra.train_id_coordinates()
    sidemic = sidemic_extra.xarray()
    return sidemic

In [ ]:
run = 193
xgm, motor_x, motor_y, motor_z, radavg, xgm2 = read_run_sync_trains(run, verbose=False)
sidemic = extract_gifs(run)


In [ ]:
rundata = RunDirectory(path=f'/gpfs/exfel/exp/SPB/202202/p003046/raw/r0{run}')

In [ ]:
rundata['SPB_IRU_MOTORS/MDL/DATA_SELECT'].keys()

In [ ]:
plt.figure()
plt.plot(rundata['SPB_IRU_MOTORS/MDL/DATA_SELECT']['SPB_IRU_INJMOV_MOTOR_Z.actualPosition.value'].xarray())
plt.show()

In [ ]:
import skimage.registration

In [ ]:
run_shift = np.zeros((sidemic.shape[0],2))
for i in np.arange(400,sidemic.shape[0],20):
    shift, error, phasediff = skimage.registration.phase_cross_correlation(sidemic[i-20], sidemic[i])
    run_shift[i] = shift

In [ ]:
plt.figure()
plt.plot(np.cumsum(run_shift[:,0])*0.0013, label='vertical shift from sidemic')
plt.plot(np.cumsum(-run_shift[:,1])*0.0013, label='horizontal shift from sidemic')
plt.plot(motor_x-motor_x[0], label='motor x')
plt.plot(motor_y-motor_y[0], label='motor y')
plt.plot(motor_z-motor_z[0], label='motor z')
plt.xlabel('Train number')
plt.ylabel('Displacement (mm)')
plt.title('Injector movement - run 193')
plt.legend()
plt.show()

In [ ]:
plt.imshow(sidemic[0])

In [ ]:
# The same trick cannot be used for the motor_x as the microscopy seems to move with the motor